In [8]:
import errant
import json

In [2]:

annotator = errant.load('en')
orig = annotator.parse('This are gramamtical sentence .')
cor = annotator.parse('This is a grammatical sentence .')
edits = annotator.annotate(orig, cor)
for e in edits:
    print(e.o_start, e.o_end, e.o_str, e.c_start, e.c_end, e.c_str, e.type)

1 2 are 1 2 is R:VERB:SVA
2 2  2 3 a M:DET
2 3 gramamtical 3 4 grammatical R:SPELL


In [3]:
from datasets import load_from_disk
import random

In [4]:
task='neutralization'
path=f'../datasets/{task}/train'
data=load_from_disk(path)
data

Dataset({
    features: ['id', 'src_tok', 'tgt_tok', 'input', 'edits', 'src_POS_tags', 'tgt_parse_tags', 'wiki_id'],
    num_rows: 53803
})

In [34]:
# Sample 10 random items
random_samples = data.shuffle(seed=42).select(range(20))
random_samples

Dataset({
    features: ['id', 'src_tok', 'tgt_tok', 'input', 'edits', 'src_POS_tags', 'tgt_parse_tags', 'wiki_id'],
    num_rows: 20
})

In [10]:
# task_prompt_path=f'../prompts/{task}.txt'
# task_instruct = open(task_prompt_path).readlines()[0].replace("\n", "")
# task_instruct

In [19]:
task_prompt_path=f'../prompts/{task}.json'
with open(task_prompt_path) as fp:
    task_prompt = json.load(fp)
task_instruct=task_prompt['task_prompt']
print(task_instruct)

Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the input sentence.


In [11]:
def get_edits(input_text, output_text):
    annotator = errant.load('en')
    orig = annotator.parse(input_text)
    cor = annotator.parse(output_text)
    edits = annotator.annotate(orig, cor)
    edit=''
    for e in edits:
        edit+=f'{e.o_start}, {e.o_end}, {e.o_str}, {e.c_start}, {e.c_end}, {e.c_str}, {e.type}\n'
    return edit

In [13]:
print(random_samples[0]['input'])
print(random_samples[0]['edits'])

it was marked by an enormous water-resources development program (which later evolved into the california aqueduct), the enactment of the university of california master plan, fair employment practices, state economic development commission, and a consumers' council.
his time in office was marked by an enormous water-resources development program (which later evolved into the california aqueduct), the enactment of the university of california master plan, fair employment practices, state economic development commission, and a consumers' council.


In [14]:
print(get_edits(random_samples[0]['input'], random_samples[0]['edits']))


0, 1, it, 0, 4, his time in office, R:OTHER



In [15]:
def get_gpt_prompt(task_instruct, input_text, suggestion, edit_content):
    prompt='You are given a pair of English sentences along with a list of '
    prompt+='atomic edits. For each edit, the first word identifies content in '
    prompt+='the source sentence that is less appropriate, while the second '
    prompt+=f'word suggests a better phrase in the target sentence. {task_instruct} Please generate a succinct explanation for each edit using the following template:\n\n'
    prompt+='The word X should be deleted/inserted/replaced by Y because ...\n\n'
    prompt+=f'###Source sentence:\n{input_text}\n\n'
    prompt+=f'###Target sentence:\n{suggestion}\n\n'
    prompt+=f'###Edits:\n{edit_content}\n'
    prompt+='###Explanation:'
    return prompt

In [35]:
i=0
edits=get_edits(random_samples[i]['input'], random_samples[i]['edits'])
prompt=get_gpt_prompt(task_instruct=task_instruct, input_text=random_samples[i]['input'], suggestion=random_samples[i]['edits'], edit_content=edits)
print(random_samples[i]['wiki_id'])
print(prompt)
    

wnc-train-131576896
You are given a pair of English sentences along with a list of atomic edits. For each edit, the first word identifies content in the source sentence that is less appropriate, while the second word suggests a better phrase in the target sentence. Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the inpu

In [36]:
i=1
edits=get_edits(random_samples[i]['input'], random_samples[i]['edits'])
prompt=get_gpt_prompt(task_instruct=task_instruct, input_text=random_samples[i]['input'], suggestion=random_samples[i]['edits'], edit_content=edits)
print(random_samples[i]['id'])
print(prompt)
    

wnc-train-165621694
You are given a pair of English sentences along with a list of atomic edits. For each edit, the first word identifies content in the source sentence that is less appropriate, while the second word suggests a better phrase in the target sentence. Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the inpu

In [37]:
i=2
edits=get_edits(random_samples[i]['input'], random_samples[i]['edits'])
prompt=get_gpt_prompt(task_instruct=task_instruct, input_text=random_samples[i]['input'], suggestion=random_samples[i]['edits'], edit_content=edits)
print(random_samples[i]['id'])
print(prompt)
    

wnc-train-417594576
You are given a pair of English sentences along with a list of atomic edits. For each edit, the first word identifies content in the source sentence that is less appropriate, while the second word suggests a better phrase in the target sentence. Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the inpu

In [38]:
i=3
edits=get_edits(random_samples[i]['input'], random_samples[i]['edits'])
prompt=get_gpt_prompt(task_instruct=task_instruct, input_text=random_samples[i]['input'], suggestion=random_samples[i]['edits'], edit_content=edits)
print(random_samples[i]['id'])
print(prompt)
    

wnc-train-169899659
You are given a pair of English sentences along with a list of atomic edits. For each edit, the first word identifies content in the source sentence that is less appropriate, while the second word suggests a better phrase in the target sentence. Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the inpu

In [39]:
i=4
edits=get_edits(random_samples[i]['input'], random_samples[i]['edits'])
prompt=get_gpt_prompt(task_instruct=task_instruct, input_text=random_samples[i]['input'], suggestion=random_samples[i]['edits'], edit_content=edits)
print(random_samples[i]['id'])
print(prompt)
    

wnc-train-207072758
You are given a pair of English sentences along with a list of atomic edits. For each edit, the first word identifies content in the source sentence that is less appropriate, while the second word suggests a better phrase in the target sentence. Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the inpu

In [40]:
i=5
edits=get_edits(random_samples[i]['input'], random_samples[i]['edits'])
prompt=get_gpt_prompt(task_instruct=task_instruct, input_text=random_samples[i]['input'], suggestion=random_samples[i]['edits'], edit_content=edits)
print(random_samples[i]['id'])
print(prompt)
    

wnc-train-692108570
You are given a pair of English sentences along with a list of atomic edits. For each edit, the first word identifies content in the source sentence that is less appropriate, while the second word suggests a better phrase in the target sentence. Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the inpu

In [41]:
i=6
edits=get_edits(random_samples[i]['input'], random_samples[i]['edits'])
prompt=get_gpt_prompt(task_instruct=task_instruct, input_text=random_samples[i]['input'], suggestion=random_samples[i]['edits'], edit_content=edits)
print(random_samples[i]['id'])
print(prompt)
    

wnc-train-693565497
You are given a pair of English sentences along with a list of atomic edits. For each edit, the first word identifies content in the source sentence that is less appropriate, while the second word suggests a better phrase in the target sentence. Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the inpu

In [42]:
i=7
edits=get_edits(random_samples[i]['input'], random_samples[i]['edits'])
prompt=get_gpt_prompt(task_instruct=task_instruct, input_text=random_samples[i]['input'], suggestion=random_samples[i]['edits'], edit_content=edits)
print(random_samples[i]['id'])
print(prompt)
    

wnc-train-70218028
You are given a pair of English sentences along with a list of atomic edits. For each edit, the first word identifies content in the source sentence that is less appropriate, while the second word suggests a better phrase in the target sentence. Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the input

In [43]:
i=8
edits=get_edits(random_samples[i]['input'], random_samples[i]['edits'])
prompt=get_gpt_prompt(task_instruct=task_instruct, input_text=random_samples[i]['input'], suggestion=random_samples[i]['edits'], edit_content=edits)
print(random_samples[i]['id'])
print(prompt)
    

wnc-train-879288389
You are given a pair of English sentences along with a list of atomic edits. For each edit, the first word identifies content in the source sentence that is less appropriate, while the second word suggests a better phrase in the target sentence. Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the inpu

In [44]:
i=9
edits=get_edits(random_samples[i]['input'], random_samples[i]['edits'])
prompt=get_gpt_prompt(task_instruct=task_instruct, input_text=random_samples[i]['input'], suggestion=random_samples[i]['edits'], edit_content=edits)
print(random_samples[i]['id'])
print(prompt)
    

wnc-train-88741599
You are given a pair of English sentences along with a list of atomic edits. For each edit, the first word identifies content in the source sentence that is less appropriate, while the second word suggests a better phrase in the target sentence. Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the input

In [ ]:
i=19
edits=get_edits(random_samples[i]['input'], random_samples[i]['edits'])
prompt=get_gpt_prompt(task_instruct=task_instruct, input_text=random_samples[i]['input'], suggestion=random_samples[i]['edits'], edit_content=edits)
print(random_samples[i]['id'])
print(prompt)
    